In [1]:
"""
@Title: 金融中的线性问题
@Author(s): Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime: 2023-10-18 21:53:24
@Description: 
"""

'\n@Title: 金融中的线性问题\n@Author(s): Stephen CUI\n@LastEditor(s): Stephen CUI\n@CreatedTime: 2023-10-18 21:53:24\n@Description: \n'

## 因子模型的多元线性回归

In [2]:
import numpy as np
import statsmodels.api as sm

In [3]:
num_periods = 9
all_values = np.array([np.random.random(8) for i in range(num_periods)])

In [4]:
y_values = all_values[:, 0]
x_values = all_values[:, 1:]
x_values = sm.add_constant(x_values) # Include the intercept
results = sm.OLS(y_values, x_values).fit()

In [5]:
results.summary()

d:\anaconda3\envs\fi\Lib\site-packages\scipy\stats\_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.963
Model:                            OLS   Adj. R-squared:                  0.704
Method:                 Least Squares   F-statistic:                     3.720
Date:                Sat, 21 Oct 2023   Prob (F-statistic):              0.380
Time:                        17:07:11   Log-Likelihood:                 14.849
No. Observations:                   9   AIC:                            -13.70
Df Residuals:                       1   BIC:                            -12.12
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.7926      0.447     -1.775      0.327      -6.466       4.881
x1             0.2035      0.191      1.068      0.479      -2.218       2.625
x2            -1.2269      0.603     -2.036      0.291      -8.882       6.429
x3             0.0609      0.233      0.261      0.837      -2.901       3.022
x4             1.2289      0.639      1.923      0.305      -6.893       9.351
x5             2.2235      0.783      2.839      0.216      -7.728      12.175
x6             0.4049      0.376      1.077      0.477      -4.373       5.183
x7             0.0318      0.346      0.092      0.942      -4.363       4.427
==============================================================================
Omnibus:                        0.955   Durbin-Watson:                   2.374
Prob(Omnibus):                  0.620   Jarque-Bera (JB):                0.666
Skew:                           0.296   Prob(JB):                        0.717
Kurtosis:                       1.807   Cond. No.                         47.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [6]:
results.params

array([-0.79262056,  0.20350966, -1.22691888,  0.06088126,  1.22892163,
        2.22348989,  0.40487267,  0.03180347])

## 线性最优化

In [7]:
import pulp

# 申明要求解的变量
x = pulp.LpVariable('x', lowBound=0)
y = pulp.LpVariable('y', lowBound=0)
# 通过文本描述和最优化的的类型来初始化问题
problem = pulp.LpProblem(
    'A_simple_maximization_objective',
    pulp.LpMaximize
)
# 允许使用 += 加入任意数量的限制条件及其描述文本
problem += 3*x + 2*y, 'The objective function'
problem += 2*x + y <= 100, 'lst constraint'
problem += x + y <= 80, '2nd constraint'
problem += x <= 40, '3rd constraint'
# 执行线性求解
problem.solve()

1

In [8]:
print('Maximization Results:')
for variable in problem.variables():
    print(variable.name, '=', variable.varValue)

Maximization Results:
x = 20.0
y = 60.0


### 整数规划

In [9]:
dealers = ['X', 'Y', 'Z']
variable_costs = {'X': 500, 'Y': 350, 'Z': 450}
fixed_costs = {'X': 4000, 'Y': 2000, 'Z': 6000}

quantities = pulp.LpVariable.dicts('quantity',
                                   dealers,
                                   lowBound=0,
                                   cat=pulp.LpInteger)
# 二进制值，表示是否为某位交易商交易
is_orders = pulp.LpVariable.dicts('orders',
                                  dealers,
                                  cat=pulp.LpBinary)

\begin{equation}
\min \sum_{i=x}^{z}\text{isOrder}_i(\text{variable cost}_i\times \text{quantity}_i+\text{fixed cost}_i)
\end{equation}

In [10]:
model = pulp.LpProblem('A_cost_minimization_problem',
                       pulp.LpMinimize)
model += sum([(variable_costs[i] * quantities[i] + fixed_costs[i])*is_orders[i] for i in dealers]), 'Minimize portfolio cost'
model += sum([quantities[i] for i in dealers]) == 150, 'Total contracts required'
model += 30 <= quantities['X'] <= 100, 'Boundary of total volume of X'
model += 30 <= quantities['Y'] <= 90, 'Boundary of total volume of Y'
model += 30 <= quantities['Z'] <= 70, 'Boundary of total volume of Z'

TypeError: Non-constant expressions cannot be multiplied

In [ ]:
model = pulp.LpProblem('A_cost_minimization_problem',
                       pulp.LpMinimize)
model += sum([variable_costs[i] * quantities[i] + fixed_costs[i] * is_orders[i] for i in dealers]), 'Minimize portfolio cost'
model += is_orders['X'] * 30 <= quantities['X'] <= is_orders['X'] * 100, 'Boundary of total volume of X'
model += is_orders['Y'] * 30 <= quantities['Y'] <= is_orders['Y'] * 90, 'Boundary of total volume of Y'
model += is_orders['Z'] * 30 <= quantities['Z'] <= is_orders['Z'] * 70, 'Boundary of total volume of Z'
model += sum([quantities[i] for i in dealers]) == 150, 'Total contracts required'
model.solve()

1

In [ ]:
print("Minimization Results:")
for variable in model.variables():
    print(variable, '=', variable.varValue)
print('Total cost:', pulp.value(model.objective))

Minimization Results:
orders_X = 0.0
orders_Y = 1.0
orders_Z = 1.0
quantity_X = 0.0
quantity_Y = 90.0
quantity_Z = 60.0
Total cost: 66500.0


## 使用矩阵解线性方程组

In [ ]:
import numpy as np

A = np.array([[2, 1, 1], 
              [1, 3, 2],
              [1, 0, 0]])
B = np.array([[4], [5], [6]])

In [ ]:
np.linalg.solve(A, B)

array([[  6.],
       [ 15.],
       [-23.]])

## LU 分解

In [ ]:
import scipy.linalg as linalg
A = np.array([
    [2, 1, 1], 
    [1, 3, 2],
    [1, 0, 0]
])
B = np.array([[4], [5], [6]])
LU = linalg.lu_factor(A)
x = linalg.lu_solve(LU, B)
x

array([[  6.],
       [ 15.],
       [-23.]])

In [ ]:
import scipy
P, L, U = scipy.linalg.lu(A)

In [ ]:
P

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [ ]:
L

array([[ 1. ,  0. ,  0. ],
       [ 0.5,  1. ,  0. ],
       [ 0.5, -0.2,  1. ]])

In [ ]:
U

array([[ 2. ,  1. ,  1. ],
       [ 0. ,  2.5,  1.5],
       [ 0. ,  0. , -0.2]])

## Cholesky分解

In [12]:
A = np.array([
    [10., -1., 2., 0.],
    [-1., 11., -1., 3.],
    [2., -1., 10., -1.],
    [0., 3., -1., 8.]
])
B = np.array([6., 25., -11., 15.])

In [14]:
L = np.linalg.cholesky(A)
L

array([[ 3.16227766,  0.        ,  0.        ,  0.        ],
       [-0.31622777,  3.3015148 ,  0.        ,  0.        ],
       [ 0.63245553, -0.24231301,  3.08889696,  0.        ],
       [ 0.        ,  0.9086738 , -0.25245792,  2.6665665 ]])

In [15]:
np.dot(L, L.T.conj())

array([[10., -1.,  2.,  0.],
       [-1., 11., -1.,  3.],
       [ 2., -1., 10., -1.],
       [ 0.,  3., -1.,  8.]])

In [18]:
y = np.linalg.solve(L, B)
x = np.linalg.solve(L.T.conj(), y)
x

array([ 1.,  2., -1.,  1.])

In [19]:
np.mat(A) * np.mat(x).T

matrix([[  6.],
        [ 25.],
        [-11.],
        [ 15.]])

## QR分解

In [24]:
import scipy
A = np.array([
    [2., 1., 1.],
    [1., 3., 2.],
    [1., 0., 0]])
B = np.array([4., 5., 6.])
Q, R = scipy.linalg.qr(A)

In [27]:
y = np.dot(Q.T, B)
x = scipy.linalg.solve(R, y)
x

array([  6.,  15., -23.])

## 使用其他矩阵代数方法求解

### Jacobi 迭代法

In [29]:
"""
Solve Ax=B with the Jacobi method
"""

import numpy as np
def jacobi(A, B, n, tol=1e-10):
    # Initializes x with zeroes with same shape and type as B
    x = np.zeros_like(B)
    
    for iter_count in range(n):
        x_new = np.zeros_like(x)
        for i in range(A.shape[0]):
            s1 = np.dot(A[i, :i], x[:i])
            s2 = np.dot(A[i, i+1:], x[i+1:])
            x_new[i] = (B[i] - s1 - s2) / A[i, i] 
        if np.allclose(x, x_new, tol):
            break
        x = x_new
    return x  

In [31]:
A = np.array([
    [10., -1., 2., 0.],
    [-1., 11., -1., 3.],
    [2., -1., 10., -1.],
    [0.0, 3., -1., 8.]])
B = np.array([6., 25., -11., 15.])
n = 25

In [33]:
x = jacobi(A, B, n)
x

array([ 1.,  2., -1.,  1.])

### Gauss-Seidel 迭代法

In [35]:
"""
Solve Ax=B with the Gauss-Seidel method
"""
def gauss(A, B, n, tol=1e-10):
    L = np.tril(A) # returns the lower triangular matrix of A
    U = A - L
    L_inv = np.linalg.inv(L)
    
    x = np.zeros_like(B)
    for i in range(n):
        Ux = np.dot(U, x)
        x_new = np.dot(L_inv, B - Ux)
        
        if np.allclose(x, x_new, tol):
            break
        x = x_new
    return x

In [37]:
A = np.array([
    [10., -1., 2., 0.],
    [-1., 11., -1., 3.],
    [2., -1., 10., -1.],
    [0.0, 3., -1., 8.]])

B = np.array([6., 25., -11., 15.])
n = 100
x = gauss(A, B, n)
print('x = ', x)

x =  [ 1.  2. -1.  1.]
